# Multi-Agent Consensus Voting System
## Proof of Concept for Vedic Astrology Platform

This notebook demonstrates the **consensus voting mechanism** for reconciling interpretations from multiple specialized LLM agents in a Vedic Astrology reading system.

### System Architecture

**6 Specialized Agents:**
1. **Integration Specialist (Prompt A)** - Synthesizes planetary positions, house lords, aspects
2. **Mathematics Validator (Prompt B)** - Validates SAV calculations, Dasha periods, degrees
3. **Risk Assessor (Prompt C)** - Evaluates doshas: Kemadruma, Kaal Sarpa, Grahan, Pitra, Daridra, Manglik
4. **Nuance Specialist (Prompt F)** - Applies D9 corrections: Paper Tiger, Neecha Bhanga, Vargottama
5. **Consensus Orchestrator** - Weighted voting + conflict resolution
6. **Quality Validator** - Final output validation

**4 Query Domains:** Career, Marriage, Health, Wealth

### Key Features:
- Weighted voting by domain type (career weights differ from marriage)
- Confidence-adjusted contributions
- Conflict detection (threshold-based, >15 points deviation)
- Multiple resolution strategies: Unanimous, Weighted Majority, Nuance Arbitration, Math Override
- Dasha-weighted scoring integration
- SAV tier interpretation (X/48 format)
- Clean JSON output ready for FastAPI

---
*Author: Randhy Paul | December 2025*
*For: Multi-Agent Orchestration Platform (LangGraph/LangFuse)*

## 1. Data Structures

In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Optional
from enum import Enum
import json

# ============================================================================
# RESOLUTION STRATEGIES (as per system design)
# ============================================================================

class ResolutionStrategy(Enum):
    """Conflict resolution strategies for multi-agent consensus."""
    UNANIMOUS = "unanimous"              # All agents agree within threshold
    WEIGHTED_MAJORITY = "weighted_majority"  # Top 3 by confidence recalculate
    NUANCE_ARBITRATION = "nuance_arbitration"  # Prompt F has priority (marriage/relationships)
    MATH_OVERRIDE = "math_override"      # Prompt B veto on calculation errors

class CertaintyLevel(Enum):
    """Agent certainty levels for interpretation."""
    HIGH = "high"      # >0.8 confidence, strong planetary indicators
    MEDIUM = "medium"  # 0.5-0.8 confidence, mixed indicators
    LOW = "low"        # <0.5 confidence, weak or conflicting indicators

# ============================================================================
# AGENT RESPONSE STRUCTURE (matches LLM output schema)
# ============================================================================

@dataclass
class AgentResponse:
    """
    Structured response from each specialized Vedic agent.
    
    Agents:
    - integration_specialist (Prompt A): Planetary synthesis
    - mathematics_validator (Prompt B): SAV/Dasha validation
    - risk_assessor (Prompt C): Dosha evaluation
    - nuance_specialist (Prompt F): D9 corrections
    """
    agent_id: str
    domain: str  # career, marriage, health, wealth
    interpretation: str
    score: float  # 0-100 potential score
    confidence: float  # 0-1 confidence level
    certainty_level: CertaintyLevel
    supporting_factors: List[str]  # Vedic factors supporting interpretation
    contradicting_factors: List[str]  # Vedic factors against interpretation
    dasha_weight: Optional[float] = None  # Dasha period influence (0-1)
    sav_score: Optional[str] = None  # SAV in X/48 format

    def to_dict(self) -> dict:
        return {
            "agent_id": self.agent_id,
            "domain": self.domain,
            "interpretation": self.interpretation,
            "score": self.score,
            "confidence": self.confidence,
            "certainty_level": self.certainty_level.value,
            "supporting_factors": self.supporting_factors,
            "contradicting_factors": self.contradicting_factors,
            "dasha_weight": self.dasha_weight,
            "sav_score": self.sav_score
        }

# ============================================================================
# CONSENSUS RESULT STRUCTURE (API output format)
# ============================================================================

@dataclass
class ConsensusResult:
    """Final consensus output for API response."""
    domain: str
    final_score: float
    final_interpretation: str
    confidence: float
    certainty_level: str
    agreement_level: str  # high, medium, low
    strategy_used: ResolutionStrategy
    conflicts_detected: int
    conflicts_resolved: int
    agent_contributions: Dict[str, float]
    dasha_adjusted: bool
    sav_tier: str  # "above_average", "average", "below_average"

    def to_dict(self) -> dict:
        return {
            "domain": self.domain,
            "final_score": round(self.final_score, 2),
            "final_interpretation": self.final_interpretation,
            "confidence": round(self.confidence, 3),
            "certainty_level": self.certainty_level,
            "agreement_level": self.agreement_level,
            "strategy_used": self.strategy_used.value,
            "conflicts_detected": self.conflicts_detected,
            "conflicts_resolved": self.conflicts_resolved,
            "agent_contributions": {k: round(v, 3) for k, v in self.agent_contributions.items()},
            "dasha_adjusted": self.dasha_adjusted,
            "sav_tier": self.sav_tier
        }

print("Data structures loaded successfully!")
print("  - ResolutionStrategy: 4 strategies defined")
print("  - CertaintyLevel: high/medium/low")
print("  - AgentResponse: includes Dasha weight + SAV score")
print("  - ConsensusResult: includes SAV tier + Dasha adjustment flag")

## 2. Consensus Engine

In [ ]:
class ConsensusEngine:
    """
    Multi-agent consensus orchestrator with weighted voting and conflict resolution.
    
    Implements the Consensus Orchestrator role from the 6-agent architecture.
    Uses domain-specific weights as per Vedic astrology requirements.
    """

    # ========================================================================
    # DOMAIN-SPECIFIC AGENT WEIGHTS
    # Each domain prioritizes different aspects of analysis
    # ========================================================================
    
    DOMAIN_WEIGHTS = {
        "career": {
            "integration_specialist": 0.30,  # Prompt A: House lords, planetary positions
            "mathematics_validator": 0.20,   # Prompt B: SAV for 10th house critical
            "risk_assessor": 0.25,           # Prompt C: Kemadruma, Daridra yoga
            "nuance_specialist": 0.25        # Prompt F: Neecha Bhanga Raja Yoga
        },
        "marriage": {
            "integration_specialist": 0.20,  # Prompt A: 7th house, Venus
            "mathematics_validator": 0.10,   # Prompt B: Less weight on math
            "risk_assessor": 0.30,           # Prompt C: Manglik, Kaal Sarpa critical
            "nuance_specialist": 0.40        # Prompt F: D9 analysis most important
        },
        "health": {
            "integration_specialist": 0.25,  # Prompt A: 6th, 8th house analysis
            "mathematics_validator": 0.15,   # Prompt B: Dasha timing for health
            "risk_assessor": 0.35,           # Prompt C: Grahan, Pitra dosha
            "nuance_specialist": 0.25        # Prompt F: Combustion corrections
        },
        "wealth": {
            "integration_specialist": 0.25,  # Prompt A: 2nd, 11th house lords
            "mathematics_validator": 0.30,   # Prompt B: SAV critical for wealth
            "risk_assessor": 0.25,           # Prompt C: Daridra yoga detection
            "nuance_specialist": 0.20        # Prompt F: Paper Tiger effect
        }
    }

    # Conflict detection threshold (points deviation from mean)
    CONFLICT_THRESHOLD = 15.0
    
    # SAV tier boundaries (out of 48)
    SAV_TIERS = {
        "excellent": 35,    # 35+ = excellent
        "above_average": 30, # 30-34 = above average
        "average": 25,       # 25-29 = average
        "below_average": 0   # <25 = below average
    }

    def __init__(self):
        self.resolution_log = []

    def calculate_consensus(self, responses: List[AgentResponse], domain: str) -> ConsensusResult:
        """
        Main consensus calculation with Dasha-weighted scoring.
        
        Steps:
        1. Apply domain-specific weights to each agent
        2. Adjust for Dasha influence if provided
        3. Detect conflicts (>15 points from mean)
        4. Apply resolution strategy
        5. Calculate final confidence and interpretation
        """
        weights = self.DOMAIN_WEIGHTS.get(domain, self.DOMAIN_WEIGHTS["career"])

        # Step 1: Calculate weighted scores with Dasha adjustment
        weighted_sum = 0.0
        weight_total = 0.0
        dasha_adjusted = False
        sav_scores = []

        for response in responses:
            agent_weight = weights.get(response.agent_id, 0.25)
            
            # Apply Dasha weight if available (multiplier on confidence)
            effective_confidence = response.confidence
            if response.dasha_weight is not None:
                effective_confidence *= (0.5 + 0.5 * response.dasha_weight)
                dasha_adjusted = True
            
            effective_weight = agent_weight * effective_confidence
            weighted_sum += response.score * effective_weight
            weight_total += effective_weight
            
            # Collect SAV scores for tier calculation
            if response.sav_score:
                try:
                    sav_num = int(response.sav_score.split('/')[0])
                    sav_scores.append(sav_num)
                except:
                    pass

        initial_score = weighted_sum / weight_total if weight_total > 0 else 50.0

        # Step 2: Calculate SAV tier
        avg_sav = sum(sav_scores) / len(sav_scores) if sav_scores else 28
        sav_tier = self._get_sav_tier(avg_sav)

        # Step 3: Detect conflicts
        conflicts = self._detect_conflicts(responses, initial_score)

        # Step 4: Resolve conflicts
        if conflicts:
            final_score, strategy = self._resolve_conflicts(responses, conflicts, initial_score, domain)
        else:
            final_score = initial_score
            strategy = ResolutionStrategy.UNANIMOUS

        # Step 5: Calculate agreement level
        score_range = max(r.score for r in responses) - min(r.score for r in responses)
        agreement_level = "high" if score_range <= 10 else ("medium" if score_range <= 20 else "low")

        # Step 6: Final confidence with agreement adjustment
        avg_confidence = sum(r.confidence for r in responses) / len(responses)
        agreement_bonus = 0.1 if agreement_level == "high" else (-0.1 if agreement_level == "low" else 0)
        final_confidence = min(1.0, max(0.0, avg_confidence + agreement_bonus))
        
        # Determine certainty level
        certainty = "high" if final_confidence > 0.8 else ("medium" if final_confidence > 0.5 else "low")

        return ConsensusResult(
            domain=domain,
            final_score=final_score,
            final_interpretation=self._generate_interpretation(final_score, agreement_level, domain, sav_tier),
            confidence=final_confidence,
            certainty_level=certainty,
            agreement_level=agreement_level,
            strategy_used=strategy,
            conflicts_detected=len(conflicts),
            conflicts_resolved=len(conflicts),
            agent_contributions={r.agent_id: round(weights.get(r.agent_id, 0.25) * r.confidence, 3) for r in responses},
            dasha_adjusted=dasha_adjusted,
            sav_tier=sav_tier
        )

    def _get_sav_tier(self, sav_value: float) -> str:
        """Classify SAV score into tier."""
        if sav_value >= 35:
            return "excellent"
        elif sav_value >= 30:
            return "above_average"
        elif sav_value >= 25:
            return "average"
        else:
            return "below_average"

    def _detect_conflicts(self, responses: List[AgentResponse], mean_score: float) -> List[AgentResponse]:
        """Detect agents whose scores deviate significantly from mean."""
        return [r for r in responses if abs(r.score - mean_score) > self.CONFLICT_THRESHOLD]

    def _resolve_conflicts(self, responses: List[AgentResponse], conflicts: List[AgentResponse], 
                          initial_score: float, domain: str) -> tuple:
        """
        Apply resolution strategy based on conflict type and domain.
        
        Strategies:
        - NUANCE_ARBITRATION: Prompt F priority for marriage/relationships
        - MATH_OVERRIDE: Prompt B veto on calculation errors
        - WEIGHTED_MAJORITY: Top 3 by confidence recalculate
        """
        
        # Check for math validator override (calculation error detected)
        math_conflict = next((c for c in conflicts if c.agent_id == "mathematics_validator"), None)
        if math_conflict and "calculation_error" in [f.lower() for f in math_conflict.contradicting_factors]:
            self.resolution_log.append({
                "strategy": "MATH_OVERRIDE", 
                "reason": "Mathematics Validator detected calculation error",
                "agent": "mathematics_validator"
            })
            return math_conflict.score, ResolutionStrategy.MATH_OVERRIDE

        # Check for nuance arbitration in marriage/relationship domains
        nuance_conflict = next((c for c in conflicts if c.agent_id == "nuance_specialist"), None)
        if nuance_conflict and domain in ["marriage", "health"]:
            self.resolution_log.append({
                "strategy": "NUANCE_ARBITRATION", 
                "reason": f"{domain.title()} domain prioritizes D9/Navamsha analysis",
                "agent": "nuance_specialist"
            })
            # Blend: 60% nuance, 40% initial consensus
            return 0.6 * nuance_conflict.score + 0.4 * initial_score, ResolutionStrategy.NUANCE_ARBITRATION

        # Default: weighted majority using top 3 by confidence
        sorted_responses = sorted(responses, key=lambda r: r.confidence, reverse=True)[:3]
        recalc_sum = sum(r.score * r.confidence for r in sorted_responses)
        recalc_weight = sum(r.confidence for r in sorted_responses)
        
        self.resolution_log.append({
            "strategy": "WEIGHTED_MAJORITY", 
            "reason": "Using top 3 agents by confidence for recalculation",
            "agents": [r.agent_id for r in sorted_responses]
        })
        return recalc_sum / recalc_weight, ResolutionStrategy.WEIGHTED_MAJORITY

    def _generate_interpretation(self, score: float, agreement: str, domain: str, sav_tier: str) -> str:
        """Generate human-readable interpretation with Vedic context."""
        
        # Score tier
        if score >= 80:
            potential = "excellent"
            outlook = "highly favorable"
        elif score >= 65:
            potential = "good"
            outlook = "favorable with some considerations"
        elif score >= 50:
            potential = "moderate"
            outlook = "mixed indicators requiring careful timing"
        else:
            potential = "challenging"
            outlook = "requires remedial measures and careful Dasha analysis"
        
        # Domain-specific interpretation
        domain_context = {
            "career": f"Professional growth shows {potential} potential. SAV tier: {sav_tier}.",
            "marriage": f"Relationship compatibility indicates {outlook}. D9 analysis critical.",
            "health": f"Vitality indicators show {potential} baseline. Monitor Dasha transitions.",
            "wealth": f"Financial prospects appear {outlook}. SAV tier: {sav_tier}."
        }
        
        base = domain_context.get(domain, f"{domain.title()} shows {potential} potential.")
        return f"{base} Agent agreement: {agreement}. Score: {score:.1f}/100."

print("Consensus Engine loaded!")
print("  - 4 domains configured: career, marriage, health, wealth")
print("  - Domain-specific agent weights defined")
print("  - SAV tier calculation integrated")
print("  - Dasha-weighted scoring enabled")
print("  - 4 resolution strategies available")

## 3. Test Cases

In [ ]:
# ============================================================================
# TEST CASE 1: CAREER DOMAIN
# Expected: HIGH AGREEMENT (all agents within 10 points)
# Demonstrates: Unanimous consensus, SAV tier calculation, Dasha weighting
# ============================================================================

def get_career_responses():
    """
    Simulated responses for CAREER domain query.
    
    Chart scenario: Strong 10th house with Jupiter aspect, 
    currently running Jupiter Mahadasha (favorable for career).
    SAV for 10th house: 32/48 (above average)
    No major career-blocking doshas present.
    """
    return [
        AgentResponse(
            agent_id="integration_specialist",
            domain="career",
            interpretation="10th lord well-placed in Kendra. Jupiter aspects 10th house from 2nd. Sun dignified in 10th.",
            score=78.5,
            confidence=0.87,
            certainty_level=CertaintyLevel.HIGH,
            supporting_factors=[
                "Jupiter aspects 10th house (5th aspect)",
                "10th lord in Kendra (angular house)",
                "Sun dignified - leadership potential"
            ],
            contradicting_factors=[],
            dasha_weight=0.85,  # Jupiter Mahadasha favorable
            sav_score="32/48"
        ),
        AgentResponse(
            agent_id="mathematics_validator",
            domain="career",
            interpretation="SAV 32/48 for 10th house confirms above-average career potential. Dasha balance: 14 years remaining.",
            score=75.0,
            confidence=0.95,
            certainty_level=CertaintyLevel.HIGH,
            supporting_factors=[
                "SAV 32/48 (above average threshold 30)",
                "Jupiter Mahadasha 14 years balance",
                "No combustion affecting career significators"
            ],
            contradicting_factors=[],
            dasha_weight=0.85,
            sav_score="32/48"
        ),
        AgentResponse(
            agent_id="risk_assessor",
            domain="career",
            interpretation="No Kemadruma yoga. No Daridra yoga. Saturn well-placed for sustained effort.",
            score=81.0,
            confidence=0.82,
            certainty_level=CertaintyLevel.HIGH,
            supporting_factors=[
                "Moon not isolated - Kemadruma absent",
                "No Daridra yoga (2nd/11th lords not afflicted)",
                "Saturn in upachaya - grows with time"
            ],
            contradicting_factors=[],
            dasha_weight=0.85,
            sav_score="32/48"
        ),
        AgentResponse(
            agent_id="nuance_specialist",
            domain="career",
            interpretation="Neecha Bhanga Raja Yoga for Mercury elevates communication skills. D9 confirms career strength.",
            score=76.0,
            confidence=0.79,
            certainty_level=CertaintyLevel.MEDIUM,
            supporting_factors=[
                "Neecha Bhanga Raja Yoga active (Mercury)",
                "D9 10th lord in own sign",
                "No Paper Tiger effect - D1 promises confirmed in D9"
            ],
            contradicting_factors=[],
            dasha_weight=0.85,
            sav_score="32/48"
        )
    ]

# ============================================================================
# TEST CASE 2: MARRIAGE DOMAIN  
# Expected: CONFLICT DETECTED (Nuance Specialist diverges by >15 points)
# Demonstrates: Conflict resolution via NUANCE_ARBITRATION
# ============================================================================

def get_marriage_responses():
    """
    Simulated responses for MARRIAGE domain query.
    
    Chart scenario: Venus strong in D1 but D9 shows major upgrade.
    Manglik dosha present but cancelled.
    SAV for 7th house: 28/48 (average)
    Nuance specialist finds D9 Venus exalted - should trigger arbitration.
    """
    return [
        AgentResponse(
            agent_id="integration_specialist",
            domain="marriage",
            interpretation="Venus in own sign in 7th. 7th lord aspects 7th. Marriage significators moderately strong.",
            score=68.0,
            confidence=0.85,
            certainty_level=CertaintyLevel.MEDIUM,
            supporting_factors=[
                "Venus in Taurus (own sign)",
                "7th lord aspects 7th house",
                "Jupiter aspects 7th (benefic influence)"
            ],
            contradicting_factors=[
                "Saturn also aspects 7th (delay indicator)"
            ],
            dasha_weight=0.70,  # Venus Antardasha - relevant
            sav_score="28/48"
        ),
        AgentResponse(
            agent_id="mathematics_validator",
            domain="marriage",
            interpretation="SAV 28/48 for 7th house - average. Venus Antardasha balance: 2.5 years.",
            score=58.0,
            confidence=0.92,
            certainty_level=CertaintyLevel.HIGH,
            supporting_factors=[
                "SAV 28/48 (within average range 25-29)",
                "Venus Antardasha active"
            ],
            contradicting_factors=[
                "Mars at 28 degrees - late degree consideration"
            ],
            dasha_weight=0.70,
            sav_score="28/48"
        ),
        AgentResponse(
            agent_id="risk_assessor",
            domain="marriage",
            interpretation="Manglik dosha present but CANCELLED by Jupiter in Kendra. Kaal Sarpa absent.",
            score=62.0,
            confidence=0.88,
            certainty_level=CertaintyLevel.MEDIUM,
            supporting_factors=[
                "Manglik cancellation applies (Jupiter in Kendra)",
                "No Kaal Sarpa dosha",
                "No Pitra dosha affecting 7th"
            ],
            contradicting_factors=[
                "Mars in 7th creates initial Manglik concern"
            ],
            dasha_weight=0.70,
            sav_score="28/48"
        ),
        AgentResponse(
            agent_id="nuance_specialist",
            domain="marriage",
            interpretation="D9 UPGRADE: Venus EXALTED in Navamsha Pisces. Vargottama Moon. D1 average but D9 excellent.",
            score=88.0,  # DIVERGENT - triggers conflict detection
            confidence=0.78,
            certainty_level=CertaintyLevel.HIGH,
            supporting_factors=[
                "Venus exalted in D9 (Pisces)",
                "Vargottama Moon - emotional stability",
                "D9 7th lord in good dignity",
                "Paper Tiger REVERSED - D9 stronger than D1"
            ],
            contradicting_factors=[
                "D1 shows only average potential"
            ],
            dasha_weight=0.70,
            sav_score="28/48"
        )
    ]

# ============================================================================
# TEST CASE 3: HEALTH DOMAIN
# Expected: MEDIUM AGREEMENT with Grahan dosha consideration
# ============================================================================

def get_health_responses():
    """
    Simulated responses for HEALTH domain query.
    
    Chart scenario: Grahan dosha present (Sun-Rahu conjunction).
    6th house has mixed influences.
    """
    return [
        AgentResponse(
            agent_id="integration_specialist",
            domain="health",
            interpretation="Lagna lord strong. 6th house has Saturn - chronic but manageable conditions.",
            score=65.0,
            confidence=0.80,
            certainty_level=CertaintyLevel.MEDIUM,
            supporting_factors=[
                "Lagna lord in Kendra",
                "8th house empty - no major surgery indicators"
            ],
            contradicting_factors=[
                "Saturn in 6th - chronic conditions possible"
            ],
            dasha_weight=0.60,
            sav_score="26/48"
        ),
        AgentResponse(
            agent_id="mathematics_validator",
            domain="health",
            interpretation="SAV 26/48 for 6th house. Rahu-Ketu axis does not afflict health houses directly.",
            score=62.0,
            confidence=0.90,
            certainty_level=CertaintyLevel.HIGH,
            supporting_factors=[
                "SAV within average range",
                "No direct Rahu-Ketu on 1-6-8 axis"
            ],
            contradicting_factors=[],
            dasha_weight=0.60,
            sav_score="26/48"
        ),
        AgentResponse(
            agent_id="risk_assessor",
            domain="health",
            interpretation="GRAHAN DOSHA present: Sun-Rahu conjunction in 5th. Monitor during Rahu periods.",
            score=55.0,
            confidence=0.85,
            certainty_level=CertaintyLevel.MEDIUM,
            supporting_factors=[],
            contradicting_factors=[
                "Grahan dosha (Sun-Rahu) affecting vitality",
                "Rahu periods require health vigilance"
            ],
            dasha_weight=0.60,
            sav_score="26/48"
        ),
        AgentResponse(
            agent_id="nuance_specialist",
            domain="health",
            interpretation="D9 Lagna lord strong. Grahan dosha mitigated by Jupiter aspect in D9.",
            score=68.0,
            confidence=0.75,
            certainty_level=CertaintyLevel.MEDIUM,
            supporting_factors=[
                "D9 shows improved health indicators",
                "Jupiter aspects Sun in D9 - Grahan mitigation"
            ],
            contradicting_factors=[],
            dasha_weight=0.60,
            sav_score="26/48"
        )
    ]

print("Test cases ready!")
print("  - Career: High agreement expected (Unanimous)")
print("  - Marriage: Conflict expected (Nuance Arbitration)")
print("  - Health: Medium agreement (Grahan dosha scenario)")

## 4. Run Tests

In [ ]:
engine = ConsensusEngine()

print("="*70)
print("TEST 1: CAREER DOMAIN")
print("Expected: High Agreement -> Unanimous Strategy")
print("="*70)

career_responses = get_career_responses()
print("\nAgent Responses (4 Specialized Agents):")
print("-" * 50)
for r in career_responses:
    print(f"  [{r.agent_id}]")
    print(f"    Score: {r.score} | Confidence: {r.confidence} | SAV: {r.sav_score}")
    print(f"    Interpretation: {r.interpretation[:60]}...")
    print()

result1 = engine.calculate_consensus(career_responses, "career")
print("\n" + "="*50)
print("CONSENSUS RESULT - CAREER")
print("="*50)
print(json.dumps(result1.to_dict(), indent=2))

In [ ]:
print("\n" + "="*70)
print("TEST 2: MARRIAGE DOMAIN")
print("Expected: Conflict Detected -> Nuance Arbitration Strategy")
print("="*70)

marriage_responses = get_marriage_responses()
print("\nAgent Responses (4 Specialized Agents):")
print("-" * 50)
for r in marriage_responses:
    print(f"  [{r.agent_id}]")
    print(f"    Score: {r.score} | Confidence: {r.confidence} | SAV: {r.sav_score}")
    print(f"    Interpretation: {r.interpretation[:60]}...")
    if r.contradicting_factors:
        print(f"    WARNING: {r.contradicting_factors[0]}")
    print()

result2 = engine.calculate_consensus(marriage_responses, "marriage")
print("\n" + "="*50)
print("CONSENSUS RESULT - MARRIAGE")
print("="*50)
print(json.dumps(result2.to_dict(), indent=2))

if engine.resolution_log:
    print("\n" + "-"*50)
    print("CONFLICT RESOLUTION LOG:")
    print("-"*50)
    for log in engine.resolution_log:
        print(f"  Strategy: {log['strategy']}")
        print(f"  Reason: {log['reason']}")
        if 'agent' in log:
            print(f"  Deciding Agent: {log['agent']}")

In [ ]:
print("\n" + "="*70)
print("TEST 3: HEALTH DOMAIN")
print("Expected: Medium Agreement with Grahan Dosha consideration")
print("="*70)

# Reset resolution log for clean test
engine.resolution_log = []

health_responses = get_health_responses()
print("\nAgent Responses (4 Specialized Agents):")
print("-" * 50)
for r in health_responses:
    print(f"  [{r.agent_id}]")
    print(f"    Score: {r.score} | Confidence: {r.confidence} | SAV: {r.sav_score}")
    print(f"    Interpretation: {r.interpretation[:60]}...")
    if r.contradicting_factors:
        for cf in r.contradicting_factors:
            print(f"    RISK: {cf}")
    print()

result3 = engine.calculate_consensus(health_responses, "health")
print("\n" + "="*50)
print("CONSENSUS RESULT - HEALTH")
print("="*50)
print(json.dumps(result3.to_dict(), indent=2))

## 5. Summary & Production Readiness

### Features Demonstrated

| Feature | Status | Description |
|---------|--------|-------------|
| 4 Specialized Agents | OK | Integration, Math, Risk, Nuance (Prompts A,B,C,F) |
| 4 Query Domains | OK | Career, Marriage, Health, Wealth |
| Domain-specific Weights | OK | Each domain prioritizes different agents |
| SAV Tier Calculation | OK | X/48 format with tier classification |
| Dasha-weighted Scoring | OK | Mahadasha/Antardasha influence on confidence |
| Conflict Detection | OK | >15 points deviation triggers resolution |
| 4 Resolution Strategies | OK | Unanimous, Weighted Majority, Nuance Arbitration, Math Override |
| Vedic Terminology | OK | Doshas, Yogas, D9 corrections integrated |
| JSON API Output | OK | Clean structure ready for FastAPI |

### Vedic Concepts Implemented

**Doshas Evaluated (Risk Assessor):**
- Kemadruma Yoga (Moon isolation)
- Daridra Yoga (wealth blocking)
- Manglik Dosha (Mars affliction)
- Kaal Sarpa Dosha (Rahu-Ketu axis)
- Grahan Dosha (Sun/Moon eclipse)
- Pitra Dosha (ancestral karma)

**D9 Corrections (Nuance Specialist):**
- Paper Tiger Effect (D1 promises not confirmed in D9)
- Neecha Bhanga Raja Yoga (debilitation cancellation)
- Vargottama (same sign in D1 and D9)

### Next Steps for Production

1. **LangGraph Integration**: Convert ConsensusEngine to LangGraph state machine
2. **LLM Agent Binding**: Connect to actual Claude/GPT agents with prompts A,B,C,F
3. **pyswisseph**: Integrate for real astronomical calculations
4. **LangFuse Tracing**: Add observability for agent interactions
5. **FastAPI Wrapper**: Expose as REST API with Pydantic models

In [ ]:
# Final Summary Table
print("="*70)
print("MULTI-AGENT CONSENSUS SYSTEM - TEST RESULTS SUMMARY")
print("="*70)
print()
print(f"{'Domain':<12} {'Score':<10} {'Strategy':<22} {'Agreement':<12} {'SAV Tier':<15}")
print("-"*70)
print(f"{'Career':<12} {result1.final_score:<10.2f} {result1.strategy_used.value:<22} {result1.agreement_level:<12} {result1.sav_tier:<15}")
print(f"{'Marriage':<12} {result2.final_score:<10.2f} {result2.strategy_used.value:<22} {result2.agreement_level:<12} {result2.sav_tier:<15}")
print(f"{'Health':<12} {result3.final_score:<10.2f} {result3.strategy_used.value:<22} {result3.agreement_level:<12} {result3.sav_tier:<15}")
print("-"*70)
print()
print("Key Observations:")
print("  1. Career: High agreement = Unanimous strategy (no conflicts)")
print("  2. Marriage: D9 upgrade detected = Nuance Arbitration applied")
print("  3. Health: Grahan dosha factored into risk assessment")
print()
print("System validates: weighted voting + conflict detection + domain-specific resolution")
print()
print("="*70)
print("READY FOR LANGGRAPH + LANGFUSE INTEGRATION")
print("="*70)

## 5. Summary

This prototype demonstrates:

| Feature | Status |
|---------|--------|
| Weighted voting by domain | OK |
| Confidence-adjusted contributions | OK |
| Conflict detection | OK |
| Resolution strategies | OK |
| Agreement level calculation | OK |
| JSON output for API | OK |

**Ready for production implementation with real LLM agents.**